# Lab 8 - the Airline Assistant Revisited


In [1]:
# imports

import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
load_dotenv(override=True)


True

In [3]:
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, witty, humorous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Behold, a tool!

In [5]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}", flush=True)
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [6]:
get_ticket_price("Sydney")

Tool get_ticket_price called for Sydney


'$2999'

In [7]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this when you need to know a ticket price.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False,
    }
}

In [8]:
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city. Call this when you need to know a ticket price.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [9]:
def call_tool_get_ticket_price(city):
    price = get_ticket_price(city)
    price_json = {"destination_city": city, "price": price}
    return json.dumps(price_json)

In [10]:
def handle_tool_call(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    results = call_tool_get_ticket_price(city)
    response = {"role": "tool", "content": results, "tool_call_id": tool_call.id}
    return response

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        messages.append(message)
        for tool_call in message.tool_calls:
            tool_response = handle_tool_call(tool_call)
            messages.append(tool_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Rome
Tool get_ticket_price called for Torino
Tool get_ticket_price called for Tokyo
